In [4]:
import pandas as pd
df = pd.read_csv("../../combined_file.csv")
df.drop(['definition', 'column_name'], axis=1, inplace=True)
df.columns

C:\Users\Franc\AppData\Local\Temp\ipykernel_21572\2358656560.py:2: DtypeWarning: Columns (0,1,2,8,9,12,13,15,17,18,19,21,22,23,24,25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../combined_file.csv")


Index(['most_recent_observation', 'most_recent_observation_type',
       'common_name', 'scientific_name', 'city', 'state',
       'longitude_coordinate', 'latitude_coordinate', 'address', 'condition',
       'height_M', 'native', 'height_binned_M',
       'diameter_breast_height_binned_CM', 'greater_metro', 'city_ID',
       'tree_ID', 'planted_date', 'retired_date', 'location_type', 'zipcode',
       'neighborhood', 'location_name', 'ward', 'district', 'overhead_utility',
       'diameter_breast_height_CM', 'percent_population'],
      dtype='object')

In [5]:
df = df[["state", "city", "common_name"]].dropna()
df1 = df.copy()
df1 = df1[df1["common_name"]!="ok to plant vacant"]
df1

,state,city,common_name
0,New Mexico,Albuquerque,Locust
1,New Mexico,Albuquerque,Oak
2,New Mexico,Albuquerque,Ash
3,New Mexico,Albuquerque,Scotch pine
4,New Mexico,Albuquerque,Honeylocust
...,...,...,...
5642805,District of Columbia,Washington DC,Redmond american linden
5642806,District of Columbia,Washington DC,Dogwood
5642807,District of Columbia,Washington DC,Flowering dogwood
5642808,District of Columbia,Washington DC,(see notes)


In [6]:
state_tree_counts = df1.groupby('state')['common_name'].count()
state_with_max_trees = state_tree_counts.idxmax()
filtered_df = df1[df1['state'] == state_with_max_trees]
tree_counts = filtered_df['common_name'].value_counts()
top_5_trees = tree_counts.head(5).index
filtered_df['common_name'] = filtered_df['common_name'].apply(lambda x: x if x in top_5_trees else 'Other')
filtered_df

C:\Users\Franc\AppData\Local\Temp\ipykernel_21572\1914998103.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['common_name'] = filtered_df['common_name'].apply(lambda x: x if x in top_5_trees else 'Other')


,state,city,common_name
2533,California,Anaheim,Mexican fan palm
2534,California,Anaheim,Mexican fan palm
2535,California,Anaheim,Mexican fan palm
2536,California,Anaheim,Mexican fan palm
2537,California,Anaheim,Mexican fan palm
...,...,...,...
5426313,California,Stockton,Southern magnolia
5426314,California,Stockton,Southern magnolia
5426315,California,Stockton,Southern magnolia
5426316,California,Stockton,Southern magnolia


In [7]:
pivot_df = pd.pivot_table(filtered_df, index=['city'], columns=['common_name'], values='state', aggfunc='count', fill_value=0)
pivot_df['Total'] = pivot_df.sum(axis=1)
pivot_df = pivot_df.drop("Total", axis=1)

In [8]:
pivot_df.to_csv("../datasets/stacked_other.csv")